### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6410,2023-11-12,Brasil Nbb,11:00,Fortaleza B.C.,Unifacisa,2.01,1.77,150.5,1.87,1.79,1.5,1.86,1.88
6411,2023-11-12,Espanha Acb,08:30,Obradoiro CAB,Baskonia,2.38,1.62,163.5,1.83,1.93,2.5,1.99,1.78
6412,2023-11-12,Espanha Acb,09:00,Gran Canaria,Granada,1.23,4.44,167.5,1.84,1.94,-10.5,2.06,1.71
6413,2023-11-12,Espanha Acb,13:00,Bilbao,Real Madrid,4.42,1.23,165.5,1.88,1.88,8.5,2.00,1.76
6414,2023-11-12,Espanha Acb,13:00,Valencia,Zaragoza,1.31,3.63,158.5,1.85,1.91,-9.5,2.08,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6597,2023-11-12,Eua Ncaa,14:30,UAB,Maryland,3.12,1.40,137.5,1.88,1.79,4.5,2.08,1.67
6598,2023-11-12,Eua Ncaa,17:00,Clemson,Davidson,1.29,3.76,138.5,1.91,1.91,-9.5,2.04,1.69
6599,2023-11-12,Libano Division 1,14:00,Champville,Mayrouba,1.37,2.90,175.5,1.85,1.81,-6.5,1.88,1.78
6600,2023-11-12,Mundo Jogos Olímpicos,18:00,Canadá F,Porto Rico F,1.04,13.00,144.5,1.83,1.95,-19.5,2.02,1.74


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,11:00,Brasil Nbb,Fortaleza B.C.,Unifacisa,150.5,1.87,0.5284,Over
6,14:00,Eua Nba,New York Knicks,Charlotte Hornets,224.5,1.86,1.0000,Over
7,17:00,Eua Nba,Brooklyn Nets,Washington Wizards,233.5,1.83,1.0000,Over
8,17:30,Eua Nba,Los Angeles Clippers,Memphis Grizzlies,224.5,1.80,1.0000,Over
9,20:00,Eua Nba,Philadelphia 76ers,Indiana Pacers,236.5,1.83,0.9997,Over
11,21:00,Eua Nba,Houston Rockets,Denver Nuggets,215.5,1.81,1.0000,Over
12,21:00,Eua Nba,New Orleans Pelicans,Dallas Mavericks,231.5,1.81,0.9999,Over
56,08:30,Espanha Leb Ouro,Estela,Real Valladolid,158.5,1.87,0.8066,Over
98,13:00,Europa Eurobasket Feminino,Ucrânia F,Portugal F,129.5,1.88,0.8374,Over


### Handicap positivo V4 (LDA)

In [9]:
# Duplicando o dataset
df_hp_v4 = jogos.copy()

df_hp_v4.loc[(df_hp_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v4.loc[(df_hp_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v4['Zebra'] = df_hp_v4['Zebra'].astype('category')

# Odds
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Casa', 'Odds'] = df_hp_v4['HA_Odds_H']
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Fora', 'Odds'] = df_hp_v4['HA_Odds_A']

df_hp_v4.reset_index(drop=True, inplace=True)
df_hp_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v4.empty:
    mdl_hp_v4 = load_model('ML/handicap_positivo/v4/v4_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v4, df_hp_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,08:30,Espanha Acb,Obradoiro CAB,Baskonia,2.5,1.99,0.5299,HA Casa
2,09:00,Espanha Acb,Gran Canaria,Granada,-10.5,1.71,0.7215,HA Fora
3,13:00,Espanha Acb,Bilbao,Real Madrid,8.5,2.00,0.9774,HA Casa
4,13:00,Espanha Acb,Valencia,Zaragoza,-9.5,1.70,0.6973,HA Fora
16,13:00,Itália Liga A,Scafati,Milano,7.5,2.05,0.9093,HA Casa
19,15:00,Itália Liga A,Reggiana,Pistoia,-9.5,1.67,0.9101,HA Fora
21,11:30,Alemanha Bbl,Alba Berlin,Braunschweig,-16.5,1.71,0.8904,HA Fora
22,11:30,Alemanha Bbl,Bamberg,Ludwigsburg,2.5,2.08,0.6269,HA Casa
24,14:30,Alemanha Bbl,Tubingen,Chemnitz,5.5,2.02,0.9853,HA Casa
25,11:00,Alemanha Pro A,Frankfurt,G. Dusseldorf,-18.5,1.72,0.7833,HA Fora


### Handicap positivo V5 (RF)

In [10]:
# Duplicando o dataset
df_hp_v5 = jogos.copy()

df_hp_v5.loc[(df_hp_v5['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v5.loc[(df_hp_v5['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v5['Zebra'] = df_hp_v5['Zebra'].astype('category')

# Odds
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Casa', 'Odds'] = df_hp_v5['HA_Odds_H']
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Fora', 'Odds'] = df_hp_v5['HA_Odds_A']

df_hp_v5.reset_index(drop=True, inplace=True)
df_hp_v5.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v5.empty:
    mdl_hp_v5 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v5, df_hp_v5)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v5_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
2,09:00,Espanha Acb,Gran Canaria,Granada,-10.5,1.71,0.6100,HA Fora
3,13:00,Espanha Acb,Bilbao,Real Madrid,8.5,2.00,0.7000,HA Casa
4,13:00,Espanha Acb,Valencia,Zaragoza,-9.5,1.70,0.7200,HA Fora
5,14:30,Espanha Acb,Barcelona,Tenerife,-11.5,1.69,0.6400,HA Fora
9,20:00,Eua Nba,Philadelphia 76ers,Indiana Pacers,-7.5,1.83,0.5100,HA Fora
...,...,...,...,...,...,...,...,...
174,08:30,Suécia Superettan,Huddinge,IK Eos,-3.5,1.72,0.5600,HA Fora
175,12:00,Suécia Superettan,Wetterbygden Stars,KFUM Fryshuset,-5.5,1.67,0.5200,HA Fora
177,06:00,Taiwan P. League+,Fubon Braves,Kaohsiung Steelers,-4.5,1.74,0.5800,HA Fora
179,09:30,Turquia Superliga,Petkim Spor,Manisa,-2.5,1.73,0.6845,HA Fora


### Handicap positivo V6 (LDA)

In [11]:
# Duplicando o dataset
df_hp_v6 = jogos.copy()

df_hp_v6.loc[(df_hp_v6['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v6.loc[(df_hp_v6['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v6['Zebra'] = df_hp_v6['Zebra'].astype('category')

# Odds
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Casa', 'Odds'] = df_hp_v6['HA_Odds_H']
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Fora', 'Odds'] = df_hp_v6['HA_Odds_A']

df_hp_v6.reset_index(drop=True, inplace=True)
df_hp_v6.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v6.empty:
    mdl_hp_v6 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v6, df_hp_v6)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v6_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
2,09:00,Espanha Acb,Gran Canaria,Granada,-10.5,1.71,0.61,HA Fora
3,13:00,Espanha Acb,Bilbao,Real Madrid,8.5,2.00,0.70,HA Casa
4,13:00,Espanha Acb,Valencia,Zaragoza,-9.5,1.70,0.72,HA Fora
5,14:30,Espanha Acb,Barcelona,Tenerife,-11.5,1.69,0.64,HA Fora
9,20:00,Eua Nba,Philadelphia 76ers,Indiana Pacers,-7.5,1.83,0.51,HA Fora
...,...,...,...,...,...,...,...,...
184,18:00,Chile Copa Do Chile,Liceo Pablo Neruda,Puente Alto,33.5,1.90,0.54,HA Casa
185,19:15,Chile Copa Do Chile,Catolica,Espanol de Talca,-30.5,1.73,0.56,HA Fora
188,17:00,Eua Ncaa,Clemson,Davidson,-9.5,1.69,0.75,HA Fora
190,18:00,Mundo Jogos Olímpicos,Canadá F,Porto Rico F,-19.5,1.74,0.55,HA Fora
